In [8]:
import pandas as pd
from scipy.stats import chi2_contingency
workflow_duration = pd.read_csv("./workflow_duration.csv", delimiter=';')



In [9]:
workflow_duration

,Total benchmark time,Billable time,Validation,Provision,Setup-and-Run,Evaluation,Tear down,ExeTime
0,22m 2s,25m,8s,2m 16s,14m 42s,29s,3m 43s,9:40
1,22m 5s,25m,6s,1m 59s,15m 8s,37s,3m 43s,20:37
2,21m 26s,24m,8s,1m 55s,14m 29s,38s,3m 34s,20:10
3,22m 21s,26m,7s,2m 26s,14m 43s,27s,3m 31s,11:21
4,21m 2s,26m,5s,1m 55s,14m 11s,1m 7s,3m 29s,10:55
5,21m 50s,24m,12s,1m 56s,14m 55s,33s,3m 38s,10:10
6,23m 6s,26m,8s,2m 37s,15m 35s,29s,3m 23s,11:21
7,21m 29s,24m,17s,1m 54s,14m 58s,34s,3m 38s,22:55
8,22m 30s,26m,5s,2m 15s,15m 11s,34s,3m 41s,21:52
9,22m 43s,25m,7s,2m 28s,15m 36s,29s,3m 33s,18:31


In [10]:
import re

def time_to_seconds(time_str):
    if pd.isna(time_str):
        return None
    minutes = re.search(r'(\d+)m', time_str)
    seconds = re.search(r'(\d+)s', time_str)
    total_seconds = 0
    if minutes:
        total_seconds += int(minutes.group(1)) * 60
    if seconds:
        total_seconds += int(seconds.group(1))
        
    return total_seconds


for workflow in ['Validation', 'Provision', 'Setup-and-Run', 'Evaluation', 'Tear down']:
    workflow_duration[workflow + ' [s]'] = workflow_duration[workflow].apply(time_to_seconds)


In [11]:
ave_time_pro_workflow = workflow_duration[['Validation [s]', 'Provision [s]', 'Setup-and-Run [s]', 'Evaluation [s]', 'Tear down [s]']].mean()
ave_time_pro_workflow

Validation [s]         8.3
Provision [s]        130.1
Setup-and-Run [s]    896.8
Evaluation [s]        35.7
Tear down [s]        215.3
dtype: float64

In [12]:
statistics = workflow_duration[['Validation [s]', 'Provision [s]', 'Setup-and-Run [s]', 'Evaluation [s]', 'Tear down [s]']].describe()
statistics


,Validation [s],Provision [s],Setup-and-Run [s],Evaluation [s],Tear down [s]
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,8.300000,130.100000,896.800000,35.700000,215.300000
std,3.653005,16.305759,27.095715,11.595497,6.515793
min,5.000000,114.000000,851.000000,27.000000,203.000000
25%,6.250000,115.250000,882.250000,29.000000,211.500000
50%,7.500000,127.000000,896.500000,33.500000,216.000000
75%,8.000000,143.500000,910.250000,36.250000,220.250000
max,17.000000,157.000000,936.000000,67.000000,223.000000


In [13]:
def execution_time_to_minutes(time_str):
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes


workflow_duration['Total bench time'] = workflow_duration['Total benchmark time'].apply(time_to_seconds)
workflow_duration['ExeTime (m.p.m)'] = workflow_duration['ExeTime'].apply(execution_time_to_minutes) #minutes past midnight
correlation = workflow_duration[['Total bench time', 'ExeTime (m.p.m)']].corr()
correlation.iloc[0, 1]


-0.05567523751784484

In [14]:
def bin_bench_dur(duration_seconds):
    if duration_seconds <= quantiles[0]:
        return 'short'
    elif quantiles[0] < duration_seconds <= quantiles[1]:
        return 'medium'
    else:
        return 'long'

def bin_exe_time(time_minutes):
    if 0 <= time_minutes <= 8 * 60:      
        return '00:00-08:00'
    elif 8 * 60 < time_minutes <= 16 * 60:  
        return '08:01-16:00'
    else:                               
        return '16:01-23:59'

workflow_duration['ExeTimeBin'] = workflow_duration['ExeTime (m.p.m)'].apply(bin_exe_time)
quantiles = workflow_duration['Total bench time'].quantile([0.33, 0.66]).tolist()
workflow_duration['BenchDurBin'] = workflow_duration['Total bench time'].apply(bin_bench_dur)
contingency_table = pd.crosstab(workflow_duration['ExeTimeBin'], workflow_duration['BenchDurBin'])
chi2, p, dof, expected = chi2_contingency(contingency_table)
chi2, p, contingency_table, expected


(0.6666666666666666,
 0.7165313105737892,
 BenchDurBin  long  medium  short
 ExeTimeBin                      
 08:01-16:00     2       2      1
 16:01-23:59     2       1      2,
 array([[2. , 1.5, 1.5],
        [2. , 1.5, 1.5]]))